In [1]:
import numpy as np
from matplotlib import pyplot
from functools import partial
from open_atmos_jupyter_utils import show_plot

from PySDM import Formulae
from PySDM.physics import si

In [2]:
formulae = Formulae(
    isotope_equilibrium_fractionation_factors="MerlivatAndNief1967+Majoube1970+Majoube1971",
    isotope_kinetic_fractionation_factors="JouzelAndMerlivat1984",
    isotope_diffusivity_ratios="Stewart1975"
)

saturation = np.linspace(.7, 1.1, 7)

# temperature_C might be replaced with temperature profile
temperature_C = np.linspace(-30, 0)
temperature = formulae.trivia.C2K(temperature_C) * si.K


In [3]:
alpha_l = formulae.isotope_equilibrium_fractionation_factors.alpha_l_2H(temperature)
alpha_l_kinetic = partial(
    formulae.isotope_kinetic_fractionation_factors.alpha_kinetic,
    alpha_equilibrium=alpha_l,
    D_ratio_heavy_to_light=formulae.isotope_diffusivity_ratios.ratio_2H_heavy_to_light(temperature)
)

alpha_i = formulae.isotope_equilibrium_fractionation_factors.alpha_i_2H(temperature)
alpha_i_kinetic = partial(
    formulae.isotope_kinetic_fractionation_factors.alpha_kinetic,
    alpha_equilibrium=alpha_i,
    D_ratio_heavy_to_light=formulae.isotope_diffusivity_ratios.ratio_2H_heavy_to_light(temperature)
)

In [4]:
for S in saturation:
    alpha = alpha_l * alpha_l_kinetic(saturation=S)
    pyplot.plot(alpha, temperature_C, 'k', alpha=0.1)
    pyplot.annotate(f'{S:.3g}', xy=(alpha[4], temperature_C[4]), rotation=55, color='k', alpha=0.5, size=10)

pyplot.plot(alpha_i_kinetic(saturation=1)*alpha_i, temperature_C, label='wrt ice')
pyplot.plot(alpha_l_kinetic(saturation=1)*alpha_l, temperature_C, label='wrt liquid')
pyplot.gca().set(
    xlabel='Fractionation factor',
    ylabel="Temperature [K]",
)
pyplot.gca().invert_yaxis()
pyplot.legend()
show_plot()
